# Chat

This example will show you how to chat with GPT, the original example is on [OpenAI Example](https://platform.openai.com/docs/guides/chat/introduction), the difference is that we will teach you how to cache the  response for exact and similar matches with **gptcache**, it will be very simple, you just need to add an extra step to initialize the cache.

Before running the example, make sure the `OPENAI_API_KEY` environment variable is set by executing `echo $OPENAI_API_KEY`. If it is not already set, it can be set by using `export OPENAI_API_KEY=YOUR_API_KEY` on Unix/Linux/MacOS systems or `set OPENAI_API_KEY=YOUR_API_KEY` on Windows systems.

Then we can learn the usage and acceleration effect of gptcache by the following code, which consists of three parts, the original openai way, the exact search and the similar search.

## OpenAI API original usage

In [1]:
import time
import openai


def response_text(openai_resp):
    return openai_resp['choices'][0]['message']['content']


question = 'what‘s chatgpt'

# OpenAI API original usage
start_time = time.time()
response = openai.ChatCompletion.create(
  model='gpt-3.5-turbo',
  messages=[
    {
        'role': 'user',
        'content': question
    }
  ],
)
print(f'Question: {question}')
print("Time consuming: {:.2f}s".format(time.time() - start_time))
print(f'Answer: {response_text(response)}\n')

Question: what‘s chatgpt
Time consuming: 4.83s
Answer: Sorry, as an AI language model, I don't have access to the internet, so I can't provide you with any information about a term ChatGPT. Could you please provide more context or clarify your question?



## OpenAI API + GPTCache, exact match cache

Initalize the cache to run GPTCache and import `openai` form `gptcache.adapter`, which will automatically set the map data manager to match the exact cahe, more details refer to [build your cache](https://gptcache.readthedocs.io/en/dev/usage.html#build-your-cache).

And if you ask ChatGPT the exact same two questions, the answer to the second question will be obtained from the cache without requesting ChatGPT again.

In [2]:
import time


def response_text(openai_resp):
    return openai_resp['choices'][0]['message']['content']

print("Cache loading.....")

# To use GPTCache, that's all you need
# -------------------------------------------------
from gptcache import cache
from gptcache.adapter import openai

cache.init()
cache.set_openai_key()
# -------------------------------------------------

question = "what's github"
for _ in range(2):
    start_time = time.time()
    response = openai.ChatCompletion.create(
      model='gpt-3.5-turbo',
      messages=[
        {
            'role': 'user',
            'content': question
        }
      ],
    )
    print(f'Question: {question}')
    print("Time consuming: {:.2f}s".format(time.time() - start_time))
    print(f'Answer: {response_text(response)}\n')

Cache loading.....
Question: what's github
Time consuming: 8.62s
Answer: GitHub is a web-based platform used by developers to store, manage, and share their code with other developers around the world. It is essentially a code hosting and collaboration platform that lets users contribute to open-source projects and share their own code with others. GitHub provides features such as version control, issue tracking, and pull requests, which allow developers to easily collaborate with others on projects. It is widely used in the software development industry and has become a standard tool for developers worldwide.

Question: what's github
Time consuming: 0.00s
Answer: GitHub is a web-based platform used by developers to store, manage, and share their code with other developers around the world. It is essentially a code hosting and collaboration platform that lets users contribute to open-source projects and share their own code with others. GitHub provides features such as version control,

## OpenAI API + GPTCache, similar search cache

Set the cache with `embedding_func` to generate embedding for the text, and `data_manager` to manager the cache data, `similarity_evaluation` to evaluate the similarities, more details refer to [build your cache](https://gptcache.readthedocs.io/en/dev/usage.html#build-your-cache).

After obtaining an answer from ChatGPT in response to several similar questions, the answers to subsequent questions can be retrieved from the cache without the need to request ChatGPT again.

In [3]:
import time


def response_text(openai_resp):
    return openai_resp['choices'][0]['message']['content']

from gptcache import cache
from gptcache.adapter import openai
from gptcache.embedding import Onnx
from gptcache.manager import CacheBase, VectorBase, get_data_manager
from gptcache.similarity_evaluation.distance import SearchDistanceEvaluation

print("Cache loading.....")

onnx = Onnx()
data_manager = get_data_manager(CacheBase("sqlite"), VectorBase("faiss", dimension=onnx.dimension))
cache.init(
    embedding_func=onnx.to_embeddings,
    data_manager=data_manager,
    similarity_evaluation=SearchDistanceEvaluation(),
    )
cache.set_openai_key()

questions = [
    "what's github",
    "can you explain what GitHub is",
    "can you tell me more about GitHub"
    "what is the purpose of GitHub"
]

for question in questions:
    start_time = time.time()
    response = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=[
            {
                'role': 'user',
                'content': question
            }
        ],
    )
    print(f'Question: {question}')
    print("Time consuming: {:.2f}s".format(time.time() - start_time))
    print(f'Answer: {response_text(response)}\n')

Cache loading.....
Question: what's github
Time consuming: 7.91s
Answer: GitHub is a web-based platform used for version control and collaboration by developers working on software projects. It provides a centralized repository for tracking changes made to code, and facilitates collaboration between developers through features such as pull requests, code reviews, and issue tracking. It is widely used for open-source software development and has become a popular platform for hosting code and collaborating on software projects.

Question: can you explain what GitHub is
Time consuming: 0.22s
Answer: GitHub is a web-based platform used for version control and collaboration by developers working on software projects. It provides a centralized repository for tracking changes made to code, and facilitates collaboration between developers through features such as pull requests, code reviews, and issue tracking. It is widely used for open-source software development and has become a popular pla